# Glow-TTS 학습

이 문서는 Glow-TTS 학습용 문서입니다.

실제 학습을 진행하시려면, 위쪽 메뉴에서 `런타임 -> 모두 실행`을 클릭하신 후,  
문서 최하단의 `Glow-TTS 학습 진행`까지 정상적으로 실행되는걸 확인해주시면 됩니다.

## 1. 할당된 GPU 확인

현재 런타임에 할당된 GPU를 확인합니다.

만약, `GPU: NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.` 라는 메시지가 출력된다면, 위쪽 메뉴에서 `런타임 -> 런타임 유형 변경`을 클릭하고 하드웨어 가속기를 `GPU`로 변경하여 저장한 후 다시 실행해주세요.

In [ ]:
! nvidia-smi

In [ ]:
import os
GPU_NAME = os.popen('nvidia-smi --query-gpu=name --format=csv,noheader').read().strip()
os.environ['GPU_NAME'] = GPU_NAME
print(f'GPU: {GPU_NAME}')

## 2. 구글 드라이브 마운트

음성합성을 위해 학습한 모델이 있는 구글 드라이브를 마운트합니다.  
마운트할 구글 드라이브 내에 다음 파일이 존재하는지 꼭 확인해주세요.

- `/Colab Notebooks/data/filelists.zip`

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 3. 필수 라이브러리 및 함수 불러오기

실행에 필요한 라이브러리 및 함수를 불러옵니다.

이 과정은 약 10분 정도 소요될 수 있습니다.

In [ ]:
!sudo apt-get update -y
!sudo apt-get install python3.8 -y
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1

In [ ]:
!sudo apt-get install python3.8-distutils
!sudo apt install python3.8-dev

In [ ]:
!python3 --version
!sudo apt install python3-pip

In [ ]:
import sys
from pathlib import Path

In [ ]:
%cd /content
!git clone --depth 1 https://github.com/sce-tts/TTS.git -b sce-tts
%cd /content/TTS

!pip install scipy==1.5 resampy pandas==1.3.4 matplotlib numpy==1.18.5
!pip install --upgrade pip
!pip uninstall numba
!pip uninstall resampy
!pip install resampy==0.3.1
!pip install numba==0.51.0
!pip install pydantic==1.10.10
!pip install -r requirements.txt
!pip install tensorboard
!pip install unidecode
!pip install blinker

!python setup.py develop

#중간에 멈추면 Y엔터 해주세요 *2

In [ ]:
!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu117

## 4. 학습할 데이터셋 불러오기

학습에 사용할 음성 데이터를 구글 드라이브에서 가져옵니다.

In [ ]:
%cd /content/TTS
!cp "/content/drive/My Drive/Colab Notebooks/data/filelists.zip" ./filelists.zip
!rm -rf ./filelists
!unzip -q filelists.zip -d ./filelists

In [ ]:
# import librosa
# audio_path = '/content/TTS/filelists/wavs/sp_10_1.wav'
# y, sr = librosa.load(audio_path)
# print(sr)


## 5. 사전 학습 데이터 불러오기

사전 학습 데이터가 구글 드라이브에 존재하지 않을 경우,  
SleepingCE Speech Dataset의 사전 학습 데이터를 내려받습니다.

In [ ]:
%cd /content/TTS
!mkdir -p "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2"
if not Path("/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/config.json").exists():
    !gdown --id 1DMKLdfZ_gzc_z0qDod6_G8fEXj0zCHvC -O glowtts-v2.zip
    !unzip -q glowtts-v2.zip -d ./
    !cp -R ./glowtts-v2/* "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/"

In [ ]:
# import json

# file_path = "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/config.json"

# with open(file_path, 'r') as file:
#     data = json.load(file)
#     print(type(data))
#     print(data)
#     print(data["sample_rate"])

In [ ]:
%cd /content/TTS
if not Path("/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/scale_stats_new.npy").exists():
    !python TTS/bin/compute_statistics.py "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/config.json" "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/scale_stats_new.npy" --data_path "/content/TTS/filelists/wavs/"

In [ ]:
with open("/content/TTS/test_sentences.txt", mode="w") as f:
    f.write("""이 문장들은 모델 학습을 위해 사용하지 않은 문장들입니다.
서울특별시 특허허가과 허가과장 허과장.
경찰청 철창살은 외철창살이고 검찰청 철창살은 쌍철창살이다.
지향을 지양으로 오기하는 일을 지양하는 언어 습관을 지향해야 한다.
그러니까 외계인이 우리 생각을 읽고 우리 생각을 우리가 다시 생각토록 해서 그 생각이 마치 우리가 생각한 것인 것처럼 속였다는 거냐?""")

## 6. TensorBoard 실행

학습 진행을 확인하기 위해 TensorBoard를 실행합니다.

최신 학습 진행 상황을 확인하려면 TensorBoard 우측 상단의 새로고침 아이콘을 클릭해주세요.

In [ ]:
!pip install zipp

In [ ]:
%load_ext tensorboard
%tensorboard --logdir="/content/drive/My Drive/Colab Notebooks/data/glowtts-v2"

## 7. Glow-TTS 학습 진행

실제 Glow-TTS 학습을 진행합니다.

학습이 정상적으로 진행되면, 이 셀은 종료되지 않고 계속 실행되는 상태를 유지합니다.

학습 진행 정도를 시각적으로 확인하시려면 바로 위의 TensorBoard를 참고해주세요.

처음 학습을 수행할 때에는 아래 셀을 그대로 실행하면 됩니다.

이전에 학습을 진행하던 모델을 이어서 학습을 진행하시려면 다음과 같이 수정한 후 실행해주세요.

- 셀에서 2 ~ 3번째 줄의 코드를 주석을 해제합니다.
- 3번째 줄의 경로를 이어서 학습을 진행할 모델의 경로로 변경합니다.  
(예시: `/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/glowtts-v2-May-31-2021_08+17AM-d897f2e`)
- 4번째 줄 아래의 코드를 제거합니다.


In [ ]:
# 파일 경로 지정
file_path = '/content/TTS/TTS/bin/train_glow_tts.py'


# 파일 열기 및 읽기
with open(file_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()

# 파일 수정
modified_lines = []
for i, line in enumerate(lines):
    if 'test_sentences_file' in line:
        modified_lines.append(line.replace('with open(config.test_sentences_file, "r") as f:', 'with open(config.test_sentences_file, "r", encoding="utf-8") as f:', 1))
    elif 'test_sentences =' in line:
        modified_lines.append(line.replace('test_sentences = [s.strip() for s in f.readlines()]', 'test_sentences = [s.strip() for s in f.readlines()]', 1))
    else:
        modified_lines.append(line)

# 수정된 내용으로 파일 저장
with open(file_path, 'w', encoding='utf-8') as f:
    f.writelines(modified_lines)

In [ ]:
# 이런 에러 뜨면 실행
# torch.cuda.OutOfMemoryError: CUDA out of memory. Tried to allocate 100.00 MiB
# import torch, gc
# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
import os

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'
print(os.environ['PYTORCH_CUDA_ALLOC_CONF'])

In [ ]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [ ]:
# 경로에 glowtts-v2-November-02-2023_04+24AM-3aa165a 이런 파일 없으면 2,3번째줄만 주석한 후 돌리기
# 경로에 있으면 2,3번째줄 주석 해제하고 전부 돌리기

%cd /content/TTS
!(python TTS/bin/train_glow_tts.py \
    --continue_path "/content/drive/MyDrive/Colab Notebooks/data/glowtts-v2/glowtts-v2-December-04-2023_10+51AM-3aa165a")
# !(python TTS/bin/train_glow_tts.py \
#     --config_path "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/config.json" \
#     --coqpit.datasets.0.path "/content/TTS/filelists"  \
#     --coqpit.audio.stats_path "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/scale_stats_new.npy"  \
#     --coqpit.test_sentences_file "/content/TTS/test_sentences.txt"  \
#     --coqpit.output_path "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/"  \
#     --coqpit.num_loader_workers 2  \
#     --coqpit.num_val_loader_workers 2  \
#     --restore_path "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/model_file.pth.tar")